In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button
import sunpy.io.fits
import matplotlib.pyplot as plt
from pathlib import Path
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import MultipleLocator
%matplotlib qt5

/home/harsh/CourseworkRepo/HMIAnalysis/env/lib/python3.7/site-packages/h5py/__init__.py:39: UserWarning: h5py is running against HDF5 1.12.2 when it was built against 1.12.1, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)


In [2]:
base_path = Path('/home/harsh/SpinorNagaraju/maps_1/stic/processed_inputs/')

In [3]:
filename = 'alignedspectra_scan1_map01_Ca.fits_stic_profiles.nc'

In [4]:
f = h5py.File(base_path / filename, 'r')
ind = np.where(f['profiles'][0, 0, 0, :, 0] != 0)[0]
wave = f['wav'][ind]
f.close()

In [5]:
def get_spectral_image(scannumber, stokes):
    f = h5py.File(base_path / filename, 'r')
    data = f['profiles'][0, scannumber, :, ind, stokes]
    f.close()
    return data

spatial_image = None
max_val = None

def get_spatial_image(wavelength_position):
    f = h5py.File(base_path / filename, 'r')
    data = f['profiles'][0, :, :, ind[wavelength_position], 0]
    f.close()
    return data

def get_stokes_params(scannumber, slitposition):
    f = h5py.File(base_path / filename, 'r')
    data = f['profiles'][()][0, scannumber, slitposition, ind, :].T
    f.close()
    return data

In [6]:
plt.imshow(get_spatial_image(300), cmap='gray', origin='lower')

In [18]:
plt.imshow(get_spectral_image(15, 3), cmap='gray', origin='lower')

In [8]:
size = plt.rcParams['lines.markersize']
scannumber = 12
slitposition = 40
waveposition = 0
stokesposition = 0

fig, axs = plt.subplots(3, 2, figsize=(10 * 1920/1080, 10))
im01a = axs[0][1].imshow(get_spatial_image(waveposition), cmap='gray', origin='lower')
im00 = axs[0][0].imshow(get_spectral_image(scannumber, stokesposition), cmap='gray', origin='lower')

iquv = get_stokes_params(scannumber, slitposition).astype(np.float64)
for i in range(1, 4):
    iquv[i] = np.round(iquv[i] * 100 / iquv[0], 2)
print (wave.shape, iquv.shape)
im10, = axs[1][0].plot(wave, iquv[0])
im10b = axs[1][0].axvline(wave[waveposition], linestyle='--', color='black', linewidth=0.5)
im11, = axs[1][1].plot(wave, iquv[1])
im20, = axs[2][0].plot(wave, iquv[2])
im21, = axs[2][1].plot(wave, iquv[3])

# axs[1][0].set_ylim(0.35, 0.8)
axs[1][0].set_ylim(np.abs(iquv[0]).min() * 0.95, np.abs(iquv[0]).max() * 1.05)
axs[1][1].set_ylim(-np.abs(iquv[1]).max() * 1.05, np.abs(iquv[1]).max() * 1.05)
axs[2][0].set_ylim(-np.abs(iquv[2]).max() * 1.05, np.abs(iquv[2]).max() * 1.05)
axs[2][1].set_ylim(-np.abs(iquv[3]).max() * 1.05, np.abs(iquv[3]).max() * 1.05)

xx = [slitposition]
yy = [scannumber]

im01b = axs[0][1].scatter(slitposition, scannumber, marker='+', color='red', linewidths=1, s=(size**2) * 8)
im01c, = axs[0][1].plot(np.ones(60) * scannumber, linestyle='--', color='black', linewidth='0.5')

axs[0][0].text(
    -0.22,
    0.5,
    'Stokes',
    transform=axs[0][0].transAxes,
    rotation=90
)

# xtick_pos = np.array([8540, 8542, 8544, 8546, 8548])
# axs[0][0].set_xticks((xtick_pos[1:] - wave[0]) / (wave[1] - wave[0]), xtick_pos[1:])
# axs[0][0].set_yticks(np.array([0, 5, 10, 15])/0.038, [0, 10, 20, 30, 40, 50])
# axs[0][1].set_xticks(np.array([0, 10, 20, 30, 40, 50, 60])/0.135, [0, 10, 20, 30, 40, 50])
# axs[0][1].set_yticks(np.array([0, 10, 20])/0.135, [0, 10, 20])

# axs[0][0].yaxis.set_minor_locator(MultipleLocator(1/0.135))

# axs[0][1].xaxis.set_minor_locator(MultipleLocator(1/0.135))
# axs[0][1].yaxis.set_minor_locator(MultipleLocator(1/0.135))

# axs[1][0].xaxis.set_minor_locator(MultipleLocator(0.5))
# axs[1][1].xaxis.set_minor_locator(MultipleLocator(0.5))
# axs[2][0].xaxis.set_minor_locator(MultipleLocator(0.5))
# axs[2][1].xaxis.set_minor_locator(MultipleLocator(0.5))

# axs[0][0].set_ylabel(r'x [arcsec]')
# axs[0][1].set_xlabel(r'x [arcsec]')
# axs[0][1].set_ylabel(r'y [arcsec]')
# axs[1][0].set_ylabel(r'$I/I_{c}$')
# axs[1][1].set_ylabel(r'$Q/I$ [%]')
# axs[2][0].set_ylabel(r'$U/I$ [%]')
# axs[2][1].set_ylabel(r'$V/I$ [%]')
# axs[2][0].set_xlabel(r'Wavelength [$\mathrm{\AA}$]')
# axs[2][1].set_xlabel(r'Wavelength [$\mathrm{\AA}$]')

stokes_axs = plt.axes([0.03, 0.685, 0.03, 0.25])
scan_axs = plt.axes([0.13, 0.96, 0.3, 0.03])
wave_axs = plt.axes([0.6, 0.96, 0.25, 0.03])

def update_stokes(val):
    global scannumber, stokesposition
    stokesposition = val
    dd = get_spectral_image(scannumber, stokesposition)
    im00.set_data(dd)
    im00.set_clim(dd.min(), dd.max())
    fig.canvas.draw_idle()

def update_scan(val):
    global scannumber, stokesposition
    scannumber = val
    dd = get_spectral_image(scannumber, stokesposition)
    im00.set_data(dd)
    im00.set_clim(dd.min(), dd.max())
    im01c.set_ydata(np.ones(440) * scannumber)
    fig.canvas.draw_idle()

def update_wave(val):
    global waveposition
    waveposition = np.argmin(np.abs(wave - val))
    dd = get_spatial_image(waveposition)
    im01a.set_data(dd)
    im01a.set_clim(dd.min(), dd.max())
    im10b.set_xdata(wave[waveposition])
    fig.canvas.draw_idle()

def on_click_event(event):
    global scannumber, slitposition, xx, yy
    ax = event.inaxes

    if ax is None or ax != axs[0][1]:
        return
    
    slitposition = np.round(event.xdata, 0).astype(np.int64)
    scannumber = np.round(event.ydata, 0).astype(np.int64)

    iquv = get_stokes_params(scannumber, slitposition).astype(np.float64)
    for i in range(1, 4):
        iquv[i] = np.round(iquv[i] * 100 / iquv[0], 2)

    im10.set_ydata(iquv[0])
    im11.set_ydata(iquv[1])
    im20.set_ydata(iquv[2])
    im21.set_ydata(iquv[3])
    
    axs[1][0].set_ylim(np.abs(iquv[0]).min() * 0.95, np.abs(iquv[0]).max() * 1.05)
    axs[1][1].set_ylim(-np.abs(iquv[1]).max() * 1.05, np.abs(iquv[1]).max() * 1.05)
    axs[2][0].set_ylim(-np.abs(iquv[2]).max() * 1.05, np.abs(iquv[2]).max() * 1.05)
    axs[2][1].set_ylim(-np.abs(iquv[3]).max() * 1.05, np.abs(iquv[3]).max() * 1.05)
    
    xx = [slitposition]
    yy = [scannumber]
    im01b.set_offsets(np.c_[xx,yy])

    fig.canvas.draw_idle()

stokes_slider = Slider(
    ax=stokes_axs,
    label='',
    valmin=0,
    valmax=3,
    valinit=0,
    valstep=1,
    orientation='vertical'
)
scan_slider = Slider(
    ax=scan_axs,
    label='',
    valmin=0,
    valmax=17,
    valinit=scannumber,
    valstep=1
)
wave_slider = Slider(
    ax=wave_axs,
    label='',
    valmin=0,
    valmax=wave.size,
    valinit=0,
    valstep=1
)
stokes_slider.on_changed(update_stokes)
scan_slider.on_changed(update_scan)
wave_slider.on_changed(update_wave)
fig.canvas.mpl_connect('button_press_event', on_click_event)

plt.subplots_adjust(left=0.1, bottom=0.05, right=0.99, top=0.95, wspace=0.2, hspace=0.2)
plt.show()

(306,) (4, 306)


In [119]:
np.arange(100, 103, 1)

array([100, 101, 102])

In [120]:
wave = np.arange(15640.899, 15640.899 + (1010 * 0.039917), 0.039917)

In [137]:
wave = np.arange(15640.899, 15640.899 + (1010 * 0.039917), 0.039917)

In [138]:
wave.shape

(1011,)

In [139]:
wave

array([15640.899   , 15640.938917, 15640.978834, ..., 15681.135336,
       15681.175253, 15681.21517 ])

In [145]:
np.arange(15640, 15640+45, 5)

array([15640, 15645, 15650, 15655, 15660, 15665, 15670, 15675, 15680])

In [153]:
(xtick_pos - 15640.899) / 0.039917

array([-22.5217326 , 102.73818173, 227.99809605, 353.25801037,
       478.51792469, 603.77783902, 729.03775334, 854.29766766,
       979.55758198])

In [203]:
splitting = 15648.459-15648.066

In [204]:
mag_field = splitting / (4.67e-13 * 15648.514**2)

In [205]:
mag_field

3436.603786618302